In [1]:
import numpy as np
from numpy.matlib import repmat
import pandas as pd
import cv2

In [2]:
im = cv2.imread("stars.jpg")

In [3]:
arr1 = np.concatenate([im[i] for i in range(len(im))])
arr2 = arr1.astype(int)

In [7]:
N = len(arr2)
rndinds = np.random.permutation(N)
Kmus = arr2[rndinds[:2]]

In [8]:
maxiters = 20

In [3]:
def calc_dist(X, Kmus):
    diff_arr1 = X - Kmus[0]
    diff_arr1_sq = diff_arr1**2
    sq_distance1 = np.sum(diff_arr1_sq, axis = 1)
    diff_arr2 = X - Kmus[1]
    diff_arr2_sq = diff_arr2**2
    sq_distance2 = np.sum(diff_arr2_sq, axis = 1)
    #diff_arr3 = X - Kmus[2]
    #diff_arr3_sq = diff_arr3**2
    #sq_distance3 = np.sum(diff_arr3_sq, axis = 1)
    return np.array([sq_distance1, sq_distance2])

In [4]:
def determineRnk(sq_dist):
    rank1 = (sq_dist[0] >= sq_dist[1])*1
    rank2 = (sq_dist[1] > sq_dist[0])*1
    #rank3 = ((sq_dist[2] > sq_dist[0]) & (sq_dist[2] > sq_dist[1]))*1
    return rank1, rank2

In [5]:
def recalcMus(X, Rnk):
    mean1 = np.array([np.mean(X[:,0] * Rnk[0]), np.mean(X[:,1] * Rnk[0]), np.mean(X[:,2] * Rnk[0])])
    mean2 = np.array([np.mean(X[:,0] * Rnk[1]), np.mean(X[:,1] * Rnk[1]), np.mean(X[:,2] * Rnk[1])])
    #mean3 = np.array([np.mean(X[:,0] * Rnk[2]), np.mean(X[:,1] * Rnk[2]), np.mean(X[:,2] * Rnk[2])])
    return np.array([mean1, mean2])

In [9]:
sqDmat = calc_dist(arr2, Kmus)
Rnk = determineRnk(sqDmat)
Rnk

(array([0, 0, 0, ..., 0, 0, 0]), array([1, 1, 1, ..., 1, 1, 1]))

In [10]:
for iter in range(maxiters):
    sqDmat = calc_dist(arr2, Kmus)
    Rnk = determineRnk(sqDmat)
    KmusOld = Kmus
    Kmus = recalcMus(arr2, Rnk)
    if sum(abs(KmusOld.flatten() - Kmus.flatten())) < 1e-6:
        break

In [12]:
rnk_lst = list(Rnk[0])
island = np.reshape(rnk_lst, (3734, 3000))
sharp_island = island*255
X = np.zeros((3734, 3000, 3))
X[:,:,0] = sharp_island
X[:,:,1] = sharp_island
X[:,:,2] = sharp_island

In [13]:
cv2.imwrite("sharp_star_test.jpg", X)

True

In [15]:
im.shape

(3734, 3000, 3)

In [35]:
def img_classify(im):
    arr1 = np.concatenate([im[i] for i in range(len(im))])
    arr2 = arr1.astype(int)
    N = len(arr2)
    rndinds = np.random.permutation(N)
    Kmus = arr2[rndinds[:2]]
    maxiters = 20
    sqDmat = calc_dist(arr2, Kmus)
    Rnk = determineRnk(sqDmat)
    for iter in range(maxiters):
        sqDmat = calc_dist(arr2, Kmus)
        Rnk = determineRnk(sqDmat)
        KmusOld = Kmus
        Kmus = recalcMus(arr2, Rnk)
        if sum(abs(KmusOld.flatten() - Kmus.flatten())) < 1e-6:
            break
    rnk_lst = list(Rnk[1])
    island = np.reshape(rnk_lst, (im.shape[0], im.shape[1]))
    sharp_island = island*255
    X = np.zeros((im.shape[0], im.shape[1], 3))
    X[:,:,0] = sharp_island
    X[:,:,1] = sharp_island
    X[:,:,2] = sharp_island
    cv2.imwrite("sharp_star_test.jpg", X)
    return X

In [19]:
img_classify(im)

In [36]:
## 局部 K-means 方法试探
out = np.zeros((3734, 3000, 3))
i = 0
j = 0
while i + 374 <= 3740:
    while j + 300 <= 3000:
        sub_img = im[i:i+374,j:j+300,:]
        sharp_sub_img = img_classify(sub_img)
        out[i:i+374,j:j+300,0] = sharp_sub_img[:,:,0]
        out[i:i+374,j:j+300,1] = sharp_sub_img[:,:,1]
        out[i:i+374,j:j+300,2] = sharp_sub_img[:,:,2]
        j = j + 300
    i = i + 374

In [14]:
im.shape

(3734, 3000, 3)

In [40]:
cv2.imwrite("sharp_star_test.jpg", sharp_sub_img)

True

In [39]:
sub_img = im[0:374,0:300,:]
sharp_sub_img = img_classify(sub_img)

In [41]:
cv2.imwrite("sharp_star_test1.jpg", sub_img)

True

In [6]:
sub_img = im[0:374,0:300,:]
sub_img.shape

(374, 300, 3)

In [11]:
meanR = sub_img[:, :, 0].mean()
meanG = sub_img[:, :, 1].mean()
meanB = sub_img[:, :, 2].mean()
stdR = sub_img[:, :, 0].std()
stdG = sub_img[:, :, 1].std()
stdB = sub_img[:, :, 2].std()

In [29]:
R_thres = meanR + 5*stdR
G_thres = meanG + 5*stdG
B_thres = meanB + 5*stdB

In [30]:
island = (sub_img[:, :, 0] >= R_thres) * (sub_img[:, :, 1] >= G_thres) * (sub_img[:, :, 2] >= B_thres) * 1

In [31]:
sharp_island = island*255

In [32]:
X = np.zeros((sub_img.shape[0], sub_img.shape[1], 3))
X[:,:,0] = sharp_island
X[:,:,1] = sharp_island
X[:,:,2] = sharp_island

In [33]:
cv2.imwrite("sharp_star_test2.jpg", X)

True

In [46]:
def img_sharp(sub_img):
    meanR = sub_img[:, :, 0].mean()
    meanG = sub_img[:, :, 1].mean()
    meanB = sub_img[:, :, 2].mean()
    stdR = sub_img[:, :, 0].std()
    stdG = sub_img[:, :, 1].std()
    stdB = sub_img[:, :, 2].std()
    R_thres = meanR + 5*stdR
    G_thres = meanG + 5*stdG
    B_thres = meanB + 5*stdB
    island = (sub_img[:, :, 0] >= R_thres) * (sub_img[:, :, 1] >= G_thres) * (sub_img[:, :, 2] >= B_thres) * 1
    sharp_island = island*255
    X = np.zeros((sub_img.shape[0], sub_img.shape[1], 3))
    X[:,:,0] = sharp_island
    X[:,:,1] = sharp_island
    X[:,:,2] = sharp_island
    return X

In [43]:
im = cv2.imread("stars.jpg")

In [47]:
## 局部 outlier 方法试探
out = np.zeros((3734, 3000, 3))
i = 0
j = 0
while i + 374 <= 3740:
    while j + 300 <= 3000:
        sub_img = im[i:i+374,j:j+300,:]
        sharp_sub_img = img_sharp(sub_img)
        out[i:i+374,j:j+300,0] = sharp_sub_img[:,:,0]
        out[i:i+374,j:j+300,1] = sharp_sub_img[:,:,1]
        out[i:i+374,j:j+300,2] = sharp_sub_img[:,:,2]
        j = j + 300
    i = i + 374
    j = 0

In [48]:
cv2.imwrite("sharp_star_test3.jpg", out)

True

In [49]:
islands = out[:,:,0]/255

In [62]:
islands[0:3, 150:180]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [66]:
islands[0][2999]

0.0

In [67]:
def mark_island(x, i, j):
    if i < 0 or i >= x.shape[0]:
        return
    if j < 0 or j >= x.shape[1]:
        return
    if x[i][j] == 0 or x[i][j] == 2:
        return
    if x[i][j] == 1:
        x[i][j] = 2
        mark_island(x, i+1, j)
        mark_island(x, i-1, j)
        mark_island(x, i, j+1)
        mark_island(x, i, j-1)

In [74]:
num_islands = 0
for i in range(islands.shape[0]):
    for j in range(islands.shape[1]):
        if islands[i][j] == 1:
            num_islands = num_islands + 1
            mark_island(islands, i, j)

In [75]:
num_islands

1533